In [ ]:
import pymysql
import pandas as pd
import traceback2
from datetime import datetime
import pandas as pd
# import matplotlib.pyplot as plt
from datetime import datetime, timedelta
# from sklearn.linear_model import LinearRegression
# from repository import get_symbols_close
import numpy as np

In [3]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'coinmarketcap'
    db_host = 'localhost'
    return pymysql.connect(unix_socket=None,
                                host=db_host,
                                user=db_user,
                                password=db_password,
                                db=db_name,
                                charset='latin1',
                                cursorclass=pymysql.cursors.DictCursor)

In [4]:
def get_cursor(connection):
    if connection == None:
        connection = __connect__()
    
    connection.ping(reconnect=True)
    return connection.cursor()

In [ ]:
def clean_data(df):
    index = df.index.values
    print(index)
    if 'USDT' in index:
        del df['USDT']
    if 'USDC' in index:
        del df['USDC']
    if 'DAI' in index:
        del df['DAI']
    if 'BUSD' in index:
        del df['BUSD']
    if 'HUSD' in index:
        del df['HUSD']
    if 'TUSD' in index:
        del df['TUSD']
    if 'BTC' in index:
        del df['BTC']
    data_values_df = pd.DataFrame(df.isna().sum(), columns= ['nan_count'])
    tmp_df = pd.DataFrame(df.count(), columns= ['values_count'])
    data_values_df['values_count'] = tmp_df['values_count']
    data_values_df['rows_count'] = data_values_df['values_count'] + data_values_df['nan_count']
    print(data_values_df)
    delete_coin_df = data_values_df[data_values_df['nan_count'] > (data_values_df['values_count']*.20)]
    delete_coin_df.sort_values(['nan_count'])
    symbols_del = delete_coin_df.index.tolist()
    for symbol in symbols_del:
        print(f'Deleteing Coin: {symbol}')
        del df[symbol]
    df.ffill(inplace=True)
    df = df.dropna(axis = 1)
    df.index = df.index + timedelta(seconds=60)
    return df
    
def get_todays_portfolio_balance(trades):
    balance = 0
    for symbol in trades:
        balance = balance + trades[symbol][3]
    return balance

def make_entry_symbol(df, trading_date, symbol, last_trade, fund):
    last_trade = last_trade.copy()
    todays_close = get_close_price_on(df, symbol, trading_date)
    if not len(last_trade):
        last_trade = [trading_date,todays_close,fund/todays_close,fund, 0]
        return last_trade
    last_trade[0] = trading_date
    last_trade[1] = todays_close
    last_trade[3] = last_trade[2]*todays_close
    last_trade[4] = last_trade[4] + 1
#     print(last_trade)
    return last_trade

def make_exit_symbol(df, trading_date, symbol, last_trade):
    quantity = last_trade[2]
    todays_close = get_close_price_on(df, symbol, trading_date)
    return quantity*todays_close
    
def get_not_in_list(new_list, old_list):
    not_in_new = []
    for symbol in old_list:
        if symbol not in new_list:
            not_in_new.append(symbol)            
    return not_in_new

def get_commons(symbols1, symbols2):
    symbols_list = []
    for symbol in symbols2:
        if symbol in symbols1:
            symbols_list.append(symbol)            
    return symbols_list

def get_joins(symbols1, symbols2):
    symbols_list = symbols1.copy()
    for symbol in symbols2:
        if symbol not in symbols1:
            symbols_list.append(symbol)            
    return symbols_list

def get_todays_top_n_coin(df, trade_date, top_n, order='tail'):
    data = df.loc[trade_date]
    data = pd.DataFrame(data)
    data = data.rename(columns={data.columns[0]:'C'})
    todays_close_df = pd.DataFrame(data)
    if order == 'head':
        todays_close_df = todays_close_df.sort_values(['C']).head(top_n)
    else:
        todays_close_df = todays_close_df.sort_values(['C']).tail(top_n)
    return todays_close_df.index.values.tolist()

def get_next_trading_day_as_index(start_date, hour):
    return pd.to_datetime((start_date + timedelta(days = 1) ).replace(hour=hour, minute=0, second=0).strftime("%Y %m %d %H:%M:00"))

def get_close_price_on(df, symbol, trade_date):
    data = df.loc[trade_date]
    data = pd.DataFrame(data)
    data = data.rename(columns={data.columns[0]:'C'})
    todays_close_df = pd.DataFrame(data)
    try:
        return todays_close_df.loc[symbol]['C']
    except:
        return 0

def get_qualified_coins(trade_date, factors_df, factors_count):
    factors_symbols = []
    index = 0
    for factor_df in factors_df:
        s1 = get_todays_top_n_coin(factor_df, trade_date, factors_count[index])
        index = index +1
        factors_symbols.append(s1)
    symbols = factors_symbols[0]
    for index in range(1, len(factors_symbols)):
        symbols = get_commons(symbols, factors_symbols[index])
    return symbols

def update_coins_performance(coins_performence, todays_trading_symbols):
    symbol_performance = [0, 0]
    for symbol in todays_trading_symbols:
        if symbol in coins_performence:
            symbol_performance = coins_performence[symbol]
        else:
            symbol_performance = [0, 0]
            coins_performence[symbol] = symbol_performance
        symbol_performance[0] = symbol_performance[0] + 1

def update_total_days_count_in_coin_performance(coins_performence, days_count):
    for symbol in coins_performence:
        coins_performence[symbol][1] = days_count

def plot_portfolio_performence(trade_date, df, factors_df, factors_count):    
    trades = {}
    trades['c-f'] = [trade_date, 0, 0, 0]
    last_day_trading_symbols = []
    is_first_day_trading = True
    portfolio_performence = []
    end_time = df.index.values.max()
    end_time = (end_time - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    coins_performence = {}
    days_count = 0
    while trade_date.timestamp() < end_time:
        days_count = days_count + 1
        todays_trading_symbols = get_qualified_coins(trade_date, factors_df, factors_count)
        update_coins_performance(coins_performence, todays_trading_symbols)
        if is_first_day_trading:
            is_first_day_trading = False
            if not len(todays_trading_symbols):
                trades['c-f'][3] = 100000
                continue
            fund = 100000 / len(todays_trading_symbols)
            for symbol in todays_trading_symbols:
                trade = make_entry_symbol(df, trade_date, symbol, [], fund)
                trades[symbol] = trade
#             is_first_day_trading = False
            last_day_trading_symbols = todays_trading_symbols.copy()
            continue
        tmp_trades = trades

        trades = {}
        exit_fund = tmp_trades['c-f'][3]
        todays_exit_symbols = get_not_in_list(todays_trading_symbols, last_day_trading_symbols)
        todays_common_symbols = get_commons(todays_trading_symbols, last_day_trading_symbols)
        todays_new_entry_symbols = get_not_in_list(todays_common_symbols, todays_trading_symbols)

        new_symbol = []
        for symbol_c in todays_common_symbols:
            trade = tmp_trades[symbol_c]
            if trade[4] > 5:
                exit_fund = exit_fund + make_exit_symbol(df, trade_date, symbol_c, trade)
                if symbol_c in todays_trading_symbols:
#                     print('Buying Again')
                    todays_new_entry_symbols.append(symbol_c)
                continue
            trade = make_entry_symbol(df, trade_date, symbol_c, trade, 0)
            trades[symbol_c] = trade
        
        for exit_symbol in todays_exit_symbols:
            exit_trade = tmp_trades[exit_symbol]
            fund = make_exit_symbol(df, trade_date, exit_symbol, exit_trade)
            exit_fund = exit_fund + fund
        if len(todays_new_entry_symbols):
            new_entry_fund = exit_fund/len(todays_new_entry_symbols)
            exit_fund = 0
            for entry_symbol in todays_new_entry_symbols:
                trade = make_entry_symbol(df, trade_date, entry_symbol, [], new_entry_fund)
                trades[entry_symbol] = trade
                new_symbol.append(entry_symbol)
        tmp_trades['c-f'][3] = exit_fund
        tmp_trades['c-f'][0] = trade_date
        trades['c-f'] = tmp_trades['c-f']
        if tmp_trades['c-f'][3] > 0:
            print(f"Uninvested amount as on {tmp_trades['c-f'][0]} is {tmp_trades['c-f'][3]}")
        portfolio_performence.append([get_todays_portfolio_balance(trades), trade_date])
        trade_date = get_next_trading_day_as_index(trade_date, trade_date.hour)
        last_day_trading_symbols = get_joins(todays_common_symbols, new_symbol)
    update_total_days_count_in_coin_performance(coins_performence, days_count)
    print(f'Currently Coins in Portfolio: \n {todays_trading_symbols}')
    return [portfolio_performence, todays_trading_symbols, coins_performence]


def plot_portfolio_for_config(close_df, factors_pct, factors_df):
    total_symbols_count = len(close_df.columns.values.tolist())
    factors_count = []
    for factor_pct in factors_pct:
        factors_count.append(int(total_symbols_count/100*factor_pct))

    start_time = factors_df[0].index.values.min()
    start_time_secs = (start_time - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    start_time = datetime.utcfromtimestamp(start_time_secs)
    trade_date = start_time + timedelta(days=0)
    trade_date = get_next_trading_day_as_index(trade_date, trade_date.hour)
    print(trade_date)
    portfolio = plot_portfolio_performence(trade_date, close_df, factors_df, factors_count)
    portfolio_performence = portfolio[0]
    performance_df = pd.DataFrame(portfolio_performence, columns=['balance', 'time'])
    performance_df.index = pd.DatetimeIndex(performance_df['time'])
    del performance_df['time']
    performance_df.plot(figsize=[18,10])
    plt.savefig(f'/tmp/portfolio_{"_".join(str(x) for x in factors_pct)}.png')
    
    print(f'Current Portfilio Balance: {portfolio_performence[-1][0]}')
    return portfolio[1:]

def get_rsq(x):
    X = x.values.reshape(-1, 1)
    Y = np.arange(1, len(X)+1).reshape(-1, 1)
    linear_regressor = LinearRegression(fit_intercept=True) 
    linear_regressor.fit(Y, X)
    return linear_regressor.score(Y, X)
#     return r2_score(Y, X)


def get_portfolio_version_coins(close_df, factors_df):
    # factors_df = [_7_days_pct_change_df, _days_sumed_pct_change_df, _6_hour_rsq_df, _pure_div_score]
    factors = [25, 10, 10, 99]
    coins_performance = plot_portfolio_for_config(close_df, factors, factors_df)
    coin_performance_df = pd.DataFrame.from_dict(coins_performance[1], orient='index', columns=['Occurrences', 'Total Trading Day'])
    coin_performance_df = coin_performance_df.sort_values(['Occurrences'], ascending = True)
    coin_performance_df.to_csv(f'/tmp/portfolio_{"_".join(str(x) for x in factors)}_coin_performance.csv')
    s1 = coins_performance[0]
    factors = [25, 99, 50, 7.5]
    coins_performance = plot_portfolio_for_config(close_df, factors, factors_df)
    coin_performance_df = pd.DataFrame.from_dict(coins_performance[1], orient='index', columns=['Occurrences', 'Total Trading Day'])
    coin_performance_df = coin_performance_df.sort_values(['Occurrences'], ascending = True)
    coin_performance_df.to_csv(f'/tmp/portfolio_{"_".join(str(x) for x in factors)}_coin_performance.csv')
    s2 = coins_performance[0]
    return get_joins(s1, s2)

def run_portfolio_version(close_df, factors_df, _6_hour_resampled_df, day):
    _7_days_pct_change_df = _6_hour_resampled_df.pct_change(periods=day*4)
    factors_df[0] = _7_days_pct_change_df
    return get_portfolio_version_coins(close_df, factors_df)

def get_coins_price(close_df, symbols):
    df = close_df.iloc[[-1]]
    symbols_price = f"As On {df.index.values[0]}<br>"
    for symbol in symbols:
        symbols_price = f'{symbols_price} {symbol}: ${df.iloc[0][symbol]},'
    return symbols_price

def run_algo(close_df):
    _6_hour_resampled_df = close_df.resample('6H').last()
    _6_hour_pct_change = _6_hour_resampled_df.pct_change(periods=1)
    _days_sumed_pct_change_df = _6_hour_pct_change.rolling(window = 4).sum()
    _pure_div_score = _6_hour_pct_change.applymap(lambda x : x if x < 0 else 0)
    _pure_div_score = _pure_div_score.rolling(window = 4).sum()
    _6_hour_rsq_df = _6_hour_resampled_df.rolling(window=28).apply(get_rsq) 
    factors_df = [None, _days_sumed_pct_change_df, _6_hour_rsq_df, _pure_div_score]
    # return run_portfolio_version(close_df, factors_df, _6_hour_resampled_df, 7)
    days = [3,4,5,7]
    msg = ""
    for day in days:
        msg = f'{msg}<br> {day}-day Lookback. <br>Coins Symbol in Portfolio are as below:<br>'
        symbols = run_portfolio_version(close_df, factors_df, _6_hour_resampled_df, day)
        msg = f'{msg} {get_coins_price(close_df, symbols)}<br><br>'
    return msg

In [6]:
def get_distinct_symbols():
    return ['BTC', 'ETH', 'BNB', 'USDT', 'DOT', 'ADA', 'XRP', 'LTC', 'LINK', 'BCH', 'XLM', 'UNI', 'USDC', 'DOGE', 'WBTC', 'CRO', 'AAVE', 'XEM', 'EOS', 'BSV', 'HT', 'ATOM', 'XMR', 'TRX', 'MIOTA', 'NEO', 'LUNA', 'XTZ', 'THETA', 'VET', 'FTT', 'DASH', 'SOL', 'GRT', 'AVAX', 'MKR', 'SNX', 'DAI', 'EGLD', 'ALGO', 'FIL', 'BUSD', 'SUSHI', 'COMP', 'KSM', 'DCR', 'CAKE', 'BTT', 'ETC', 'ZEC', 'RVN', 'YFI', 'LEO', 'ZIL', 'UMA', 'VGX', 'REN', 'NEXO', 'ZRX', 'CEL', 'OKB', 'WAVES', 'NEAR', 'ICX', 'ONT', 'RUNE', 'IOST', 'DGB', 'REV', 'HBAR', 'RENBTC', 'FTM', 'CELO', 'NANO', 'LRC', 'BNT', 'BAT', 'OMG', 'BTCB', 'MATIC', 'CHSB', 'ZEN', '1INCH', 'PAX', 'QTUM', 'NPXS', 'XVS', 'DODO', 'CRV', 'SC', 'BTG', 'HUSD', 'STX', 'UST', 'HOT', 'FLOW', 'RSR', 'KCS', 'ENJ', 'QNT', 'HEDG', 'OCEAN', 'TFUEL', 'MANA', 'LSK', 'EWT', 'KNC', 'GLM', 'XVG', 'SNT', 'NU', 'REP', 'AMPL', 'BAND', 'TUSD', 'CVC', 'ALPHA', 'SCRT', 'POLS', 'CHZ', 'BAL', 'HNT', 'IOTX', 'ZB', 'KAVA', 'ONE', 'CKB', 'SXP', 'ANT', 'OXT', 'BCD', 'GNO', 'AR', 'SWAP', 'VAI', 'SRM', 'FUN', 'INJ', 'ANKR', 'KMD', 'UBT', 'STRAX', 'FET', 'RIF', 'STORJ', 'VTHO', 'SFI', 'ARDR', 'POLY', 'ARK', 'STEEM', 'DENT', 'BTS', 'ROSE', 'DNT', 'UTK', 'MFT', 'NMR', 'AGI', 'TEL', 'MAID', 'BTM', 'WAN', 'MONA', 'RLC', 'GT', 'ELF', 'NOIA', 'TOMO', 'TRAC', 'RDD', 'STAKE', 'BORA', 'LOOM', 'JST', 'MX', 'PAXG', 'WAXP', 'BADGER', 'PIVX', 'IRIS', 'KIN', 'ABBC', 'REEF', 'CVT', 'PROM', 'AVA', 'AION', 'COTI', 'ETN', 'HIVE', 'POWR', 'HNS', 'PPT', 'NXS', 'SYS', 'XHV', 'YFII', 'AKRO', 'CELR', 'CTC', 'CCXX', 'DFI', 'HEX', 'INO', 'BRG', 'WBNB', 'THR', 'ZKS', 'INB', 'vBNB', 'NXM', 'BTMX', 'CIPHC', 'GNY', 'vXVS', 'STETH', 'BEST', 'IZE', 'XDC', 'ZLW', 'BCHA', 'BOTX', 'SFP', 'AMP', 'BAKE', 'HBTC', 'TTT', 'MIR', 'vBTC', 'XWC', 'SAND', 'DDX', 'XOR', 'MWC', 'KEEP', 'LIT', 'PRQ', 'RFOX', 'MATH', 'AKT', 'ORN', 'TWT', 'HTR', 'ICH', 'EAURIC', 'LRG', 'SOLO', 'EUM', 'XCM', 'NFTX', 'SKL', 'USDN', 'LINA', 'CAS', 'DRS', 'PERP', 'SUSD', 'NYE', 'MASS', 'EDG', 'HEGIC', 'FARM', 'GRN', 'JUL', 'RPL', 'AUTO', 'MBN', 'HNC', 'YUSRA', 'RGT', 'GALA', 'SAPP', 'REV', 'BNANA', 'BLCT', 'DIGG', 'TORN', 'GUSD', 'MXX', 'CREAM', 'JULD', 'BIFI', 'MRPH', 'NRG', 'ALBT', 'ATT', 'FRONT', 'LBC', 'XNC', 'QKC', 'NEST', 'XIN', 'BCN', 'STMX', 'ORC', 'ANY', 'WNXM', 'vUSDC', 'LTO', 'HARD', 'SHR', 'ID', 'LPT', 'OGN', 'IQ', 'FRAX', 'BASID', 'REQ', 'BURGER', 'BDX', 'SPARTA', 'BZRX', 'SWINGBY', 'DATA', 'GAS', 'HPT', 'ESD', 'DDIM', 'FRM', 'UOS', 'VLX', 'SUN', 'ADX', 'RAY', 'MTC', 'BOND', 'DAO', 'MVL', 'WRX', 'SOLVE', 'YOUC', 'WTC', 'PCX', 'DIA', 'UNFI', 'WIN', 'TT', 'AXEL', 'VITAE', 'TROY', 'TRB', 'PHA', 'vETH', 'DIVI', 'SNL', 'SRK', 'FIRO', 'WOO', 'AXS', 'PAID', 'ERG', 'ADK', 'DX', 'BONDLY', 'FLM', 'ORBS', 'REW', 'ELA', 'COVER', 'MTL', 'WHALE', 'DUSK', 'KP3R', 'QC', 'FXS', 'API3', 'IGNIS', 'CTK', 'ORAI', 'NULS', 'UQC', 'CVP', 'MLN', 'WICC', 'KAI', 'WOZX', 'TLOS', 'OM', 'AE', 'ATRI', 'QQQ', 'BIP', 'BEAM', 'MXC', 'POND', 'BEL', 'KLV', 'EMC2', 'ARRR', 'LON', 'TITAN', 'CTSI', 'BTRS', 'RING', 'TSHP', 'EURS', 'BTU', 'GRS', 'HXRO', 'BOA', 'TRU', 'DMCH', 'COS', 'CENNZ', 'CRE', 'TMTG', 'NIM', 'HC', 'BLZ', 'APY', 'DIP', 'vBUSD', 'RCN', 'OCTO', 'DUCK', 'LOKI', 'TNC', 'aEth', 'VSYS', 'ZAP', 'AUDIO', 'CORE', 'ZNN', 'MCO', 'DRGN', 'LAMB', 'DF', 'GHST', 'WEST', 'MTA', 'PNK', 'RAMP', 'SKY', 'CTXC', 'STPT', 'BAO', 'SPND', 'SURE', 'GXC', 'OPIUM', 'BOT', 'GLEEC', 'SWTH', 'NEBL', 'ONE', 'PNT', 'USDX', 'MUSD', 'MAPS', 'ARPA', 'CND', 'KEY', 'BRY', 'ROOM', 'SUTER', 'KDA', 'NXT', 'QSP', 'IDEX', 'UPP', 'NAS', 'VERI', 'DGD', 'COCOS', 'BNK', 'MIN', 'CUSD', 'NSBT', 'WHITE', 'VTC', 'DEXT', 'GBYTE', 'KRT', 'MEME', 'AST', 'FSN', 'REVV', 'MDT', 'MPH', 'RBC', 'WXT', 'APL', 'BHAO', 'PIB', 'MED', 'SRN', 'DAG', 'VIDT', 'EGG', 'RFR', 'FX', 'SCC', 'KAN', 'DVP', 'SPI', 'NAV', 'GRIN', 'AERGO', 'EASY', 'PERL', 'NMC', 'QBIT', 'XSN', 'PROS', 'SERO', 'TVK', 'SOUL', 'FIDA', 'SNTVT', 'DREP', 'UNISTAKE', 'FIO', 'XLT', 'LGCY', 'NKN', 'RBTC', 'ALEPH', 'AOA', 'MET', 'LGO', 'RNDR', 'XRT', 'KSP', 'PHB', 'VRSC', 'BOR', 'BPS', 'ULT', 'SBD', 'FXF', 'XSR', 'CRD', 'RFUEL', 'DAD', 'vUSDT', 'WING', 'PRE', 'EDR', 'FOR', 'DEXE', 'BMI', 'HEZ', 'STAX', 'BAC', 'DMT', 'ANW', 'ZCN', 'QRL', 'BZ', 'PAC', 'RDN', 'DEC', 'DKA', 'USDK', 'CHR', 'MLK', 'UFT', 'NEX', 'BASIC', 'VSP', 'QASH', 'HMR', 'XDB', 'PAY', 'PMA', 'PEAK', 'MBL', 'XPR', 'SALT', 'GEO', 'ASTA', 'DOCK', 'DUCATO', 'CRPT', 'GO', 'WOM', 'SOCKS', 'DG', 'MARO', 'YLD', 'MITX', 'TOP', 'DERO', 'MDA', 'ARDX', 'FST', 'DEGO', 'RKN', 'BAR', 'WABI', 'KEX', 'META', 'ZEE', 'GDC', 'DNA', 'FIS', 'HAI', 'HUM', 'DHT', 'SPC', 'vSXP', 'MITH', 'EFX', 'PICKLE', 'QTCON', 'PI', 'CCA', 'NVT', 'PZM', 'WPR', 'SENT', 'APPC', 'OAX', 'FCT', 'VID', 'BFC', 'GVT', 'MOC', 'AMO', 'AMB', 'TRUE', 'YOP', 'UBQ', 'IHF', 'FLG', 'SFT', 'GOM2', 'TXL', 'HAKKA', 'BFT', 'PAI', 'CUDOS', 'ARMOR', 'BIKI', 'ELAMA', 'AWC', 'POA', 'YF-DAI', 'ROOK', 'GARD', 'L2', 'GAL', 'MOF', 'ZEON', 'vLINK', 'LYXe', 'CRU', 'HUNT', 'PRO', 'NEW', 'MRX', 'DAC', 'OBSR', 'KEBAB', 'IDLE', 'SHROOM', 'BHD', 'CWS', 'NWC', 'ETP', 'GTO', 'LAYER', 'BRD', 'CET', 'TCT', 'SUKU', 'LOC', 'NCASH', 'DSLA', 'VITE', 'LOL', 'vLTC', 'BLOCK', 'RAZOR', 'PROPS', 'RLY', 'PTF', 'RUFF', 'BTC2', 'CBC', 'DAPP', 'VIA', 'COMBO', 'EGT', 'FNT', 'S4F', 'KDAG', 'THUGS', 'TRTT', 'SWRV', 'MIX', 'LA', 'DTR', 'FREE', 'MARK', 'GAME', 'OST', 'RINGX', 'JNT']

In [7]:
symbols = get_distinct_symbols()

In [8]:
def get_symbols_close(symbols):
    connection = __connect__()
    s = f"'{symbols[0]}'"
    for symbol in symbols:
        s = f"{s},'{symbol}'"
    sql = f"select * from coin_ohlc where symbol in ({s})"
    df = pd.read_sql(sql, connection)
    connection.close()
    return df

In [9]:
def delete_unnessecary_columns(df):
    del df['O']
    del df['H']
    del df['L']
    # del df['time']
    df = df.drop_duplicates(keep='first')
    return df

def convert_symbols_to_columns(df):
    df = df.pivot(columns='symbol', values='C')
    return df

In [11]:
df = get_symbols_close(symbols)

In [46]:
df = df.set_index(['timestamp_dt'])

In [62]:
df_o = df

In [63]:
df = delete_unnessecary_columns(df.copy())

In [68]:
df = convert_symbols_to_columns(df)

In [69]:
df = df[df.index >= datetime(2020, 12,26)]
df = clean_data(df)

['2020-12-26T01:00:00.000000000' '2020-12-26T02:00:00.000000000'
 '2020-12-26T03:00:00.000000000' ... '2021-02-22T07:00:00.000000000'
 '2021-02-22T08:00:00.000000000' '2021-02-22T09:00:00.000000000']
        nan_count  values_count  rows_count
symbol                                     
1INCH        1049           352        1401
AAVE           50          1351        1401
ABBC         1389            12        1401
ADA          1311            90        1401
ADK          1113           288        1401
...           ...           ...         ...
vLTC         1173           228        1401
vSXP         1394             7        1401
vUSDC        1400             1        1401
vUSDT        1400             1        1401
vXVS         1256           145        1401

[698 rows x 3 columns]
Deleteing Coin: 1INCH
Deleteing Coin: ABBC
Deleteing Coin: ADA
Deleteing Coin: ADK
Deleteing Coin: ADX
Deleteing Coin: AE
Deleteing Coin: AERGO
Deleteing Coin: AGI
Deleteing Coin: AION
Deleteing Coin: AKR

<ipython-input-60-2a7b7b36790a>:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'USDT' in index:
<ipython-input-60-2a7b7b36790a>:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'USDC' in index:
<ipython-input-60-2a7b7b36790a>:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'DAI' in index:
<ipython-input-60-2a7b7b36790a>:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'BUSD' in index:
<ipython-input-60-2a7b7b36790a>:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'HUSD' in index:
<ipython-input-60-2a7b7b36790a>:14: FutureWarning: elementwise comparison failed; re

Deleteing Coin: DEXT
Deleteing Coin: DF
Deleteing Coin: DFI
Deleteing Coin: DGB
Deleteing Coin: DHT
Deleteing Coin: DIA
Deleteing Coin: DIGG
Deleteing Coin: DIP
Deleteing Coin: DIVI
Deleteing Coin: DKA
Deleteing Coin: DMCH
Deleteing Coin: DMT
Deleteing Coin: DNA
Deleteing Coin: DNT
Deleteing Coin: DOCK
Deleteing Coin: DODO
Deleteing Coin: DOGE
Deleteing Coin: DOT
Deleteing Coin: DREP
Deleteing Coin: DRGN
Deleteing Coin: DRS
Deleteing Coin: DSLA
Deleteing Coin: DTR
Deleteing Coin: DUCATO
Deleteing Coin: DUCK
Deleteing Coin: DUSK
Deleteing Coin: DVP
Deleteing Coin: DX
Deleteing Coin: EASY
Deleteing Coin: EAURIC
Deleteing Coin: EDG
Deleteing Coin: EDR
Deleteing Coin: EFX
Deleteing Coin: EGG
Deleteing Coin: EGLD
Deleteing Coin: EGT
Deleteing Coin: ELA
Deleteing Coin: ELAMA
Deleteing Coin: ELF
Deleteing Coin: EMC2
Deleteing Coin: ENJ
Deleteing Coin: EOS
Deleteing Coin: ERG
Deleteing Coin: ESD
Deleteing Coin: ETC
Deleteing Coin: ETN
Deleteing Coin: ETP
Deleteing Coin: EUM
Deleteing Coin: EUR

Deleteing Coin: TWT
Deleteing Coin: TXL
Deleteing Coin: UBQ
Deleteing Coin: UBT
Deleteing Coin: UFT
Deleteing Coin: ULT
Deleteing Coin: UMA
Deleteing Coin: UNFI
Deleteing Coin: UNI
Deleteing Coin: UNISTAKE
Deleteing Coin: UOS
Deleteing Coin: UPP
Deleteing Coin: UQC
Deleteing Coin: USDC
Deleteing Coin: USDK
Deleteing Coin: USDN
Deleteing Coin: USDT
Deleteing Coin: USDX
Deleteing Coin: UST
Deleteing Coin: UTK
Deleteing Coin: VAI
Deleteing Coin: VERI
Deleteing Coin: VET
Deleteing Coin: VGX
Deleteing Coin: VIA
Deleteing Coin: VID
Deleteing Coin: VIDT
Deleteing Coin: VITAE
Deleteing Coin: VITE
Deleteing Coin: VLX
Deleteing Coin: VRSC
Deleteing Coin: VSP
Deleteing Coin: VSYS
Deleteing Coin: VTC
Deleteing Coin: VTHO
Deleteing Coin: WABI
Deleteing Coin: WAN
Deleteing Coin: WAVES
Deleteing Coin: WAXP
Deleteing Coin: WBNB
Deleteing Coin: WEST
Deleteing Coin: WHALE
Deleteing Coin: WICC
Deleteing Coin: WIN
Deleteing Coin: WING
Deleteing Coin: WNXM
Deleteing Coin: WOM
Deleteing Coin: WOO
Deleteing 

In [54]:
df[df['symbol'] == 'AAVE']

,symbol,O,H,L,C
timestamp_dt,,,,,
2020-12-26 00:59:00,AAVE,79.82,79.98,79.04,79.05
2020-12-26 01:59:00,AAVE,79.04,79.04,78.19,78.60
2020-12-26 02:59:00,AAVE,78.60,78.89,78.04,78.04
2020-12-26 03:59:00,AAVE,78.04,78.85,78.04,78.63
2020-12-26 04:59:00,AAVE,78.64,78.67,78.10,78.10
...,...,...,...,...,...
2021-02-22 04:59:00,AAVE,430.40,434.10,426.09,429.78
2021-02-22 05:59:00,AAVE,430.14,435.95,429.44,429.44
2021-02-22 06:59:00,AAVE,429.79,437.69,428.91,436.17


In [57]:
df2.to_csv('df2.csv')

In [58]:
df2.columns

Index(['1INCH', 'AAVE', 'ABBC', 'ADA', 'ADK', 'ADX', 'AE', 'AERGO', 'AGI',
       'AION',
       ...
       'vBNB', 'vBTC', 'vBUSD', 'vETH', 'vLINK', 'vLTC', 'vSXP', 'vUSDC',
       'vUSDT', 'vXVS'],
      dtype='object', name='symbol', length=698)

In [52]:
df.columns

Index(['symbol', 'O', 'H', 'L', 'C'], dtype='object')

In [70]:

df


symbol,AAVE,BCH,BIFI,BOR,BOT,BSV,BTC,BTCB,COMP,CORE,...,WBTC,XIN,XMR,XOR,YF-DAI,YFI,YFII,ZEC,aEth,vBTC
timestamp_dt,,,,,,,,,,,,,,,,,,,,,
2020-12-26 01:01:00,79.05,316.16,79.49,224.23,401.41,164.78,24611.27,19276.35,138.15,2950.67,...,24591.68,137.52,162.02,105.15,3008.79,23698.95,1486.89,62.66,588.26,385.85
2020-12-26 02:01:00,78.60,321.03,81.53,236.79,400.58,165.66,24873.31,19483.00,136.82,2964.12,...,24896.76,137.95,164.15,107.31,3024.42,23441.95,1474.15,63.82,590.88,390.07
2020-12-26 03:01:00,78.04,316.74,77.84,233.96,400.18,164.48,24823.33,19463.21,135.75,2969.45,...,24827.78,137.98,164.25,107.60,3074.84,23389.67,1467.83,63.54,590.38,389.59
2020-12-26 04:01:00,78.63,324.45,79.32,237.16,401.11,166.01,24865.27,19495.95,136.63,3001.37,...,24896.14,137.48,164.25,107.75,3135.42,23769.70,1475.96,66.11,595.55,390.39
2020-12-26 05:01:00,78.10,319.71,80.18,232.27,401.01,165.05,24829.23,19467.06,133.59,2983.08,...,24806.38,136.43,164.01,106.18,3130.51,23362.60,1455.31,65.75,595.04,389.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-22 05:01:00,429.78,679.02,1445.31,527.86,3002.69,236.94,55696.12,55538.88,435.70,5267.63,...,55537.58,185.25,237.23,647.17,4986.35,41219.40,2901.92,152.84,1849.37,1117.90
2021-02-22 06:01:00,429.44,677.58,1457.73,520.86,3002.48,238.34,55821.79,55684.55,435.47,5291.78,...,55634.04,185.38,236.49,637.40,5233.37,41572.32,2879.27,153.64,1900.85,1118.20
2021-02-22 07:01:00,436.17,690.20,1469.43,527.14,3130.87,241.17,56170.28,56145.43,443.32,5324.70,...,56053.89,187.19,242.08,615.10,5078.78,42294.49,2879.27,155.83,1880.73,1126.55
